In [1]:
_RIOT_API_KEY='RGAPI-f4f1a637-f067-477c-b698-1183d53de582'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
lol_watcher = LolWatcher(_RIOT_API_KEY)


1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

In [2]:
match_list=help.get_matchlist_by_summoner_name('souporsecret', lol_watcher)
len(match_list)
#arena_match = lol_watcher.match.by_id('na1', match_list[0])
#flex_match = lol_watcher.match.by_id('na1', match_list[4])


933

In [3]:
match_details = help.slurp_match_details(lol_watcher, match_list[0])
match_details

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,teamName,item1Name,item2Name,item3Name,item4Name,item5Name,item6Name,summoner1Name,summoner2Name,queueName
0,0,0,2,0,0,0,0,12500,13,22,...,Red,Guardian's Hammer,Blade of The Ruined King,Wit's End,Titanic Hydra,NaN,Arcane Sweeper,Flee,Flash,Arena
1,0,0,3,0,0,0,0,12500,13,33,...,Red,Guardian's Amulet,"Jak'Sho, The Protean",Spirit Visage,Force of Nature,NaN,Arcane Sweeper,Flee,Flash,Arena
2,0,0,4,0,0,0,0,12500,13,74,...,Red,Zhonya's Hourglass,Luden's Tempest,Rabadon's Deathcap,Sorcerer's Shoes,NaN,Arcane Sweeper,Flee,Flash,Arena
3,0,0,1,0,0,0,0,12500,13,35,...,Red,Guardian's Orb,Liandry's Anguish,Rylai's Crystal Scepter,Lich Bane,NaN,Arcane Sweeper,Flee,Flash,Arena
4,0,0,10,0,0,0,0,17000,18,61,...,Blue,Lich Bane,Luden's Tempest,Seraph's Embrace,Rabadon's Deathcap,Night Harvester,Arcane Sweeper,Flee,Flash,Arena
5,0,0,8,0,0,0,1,17000,18,22,...,Blue,Bloodthirster,Wit's End,Immortal Shieldbow,Rapid Firecannon,Mercury's Treads,Arcane Sweeper,Flee,Flash,Arena
6,0,0,6,0,0,0,0,17043,18,74,...,Blue,Morellonomicon,Luden's Tempest,Rite Of Ruin,Demonic Embrace,Shadowflame,Arcane Sweeper,Flee,Flash,Arena
7,0,0,4,0,0,0,0,17043,18,55,...,Blue,Atma's Reckoning,Hextech Gunblade,Riftmaker,Rabadon's Deathcap,Rite Of Ruin,Arcane Sweeper,Flee,Flash,Arena


# Match Timeline Work

In [9]:
match_timeline = help.slurp_match_timeline(lol_watcher, match_list[0], region='na1')
match_timeline

,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,totalDamageDoneToChampions,posX,posY
0,0,1,0,3,2000,0,0,0,9513,9193
1,0,2,0,3,2000,0,0,0,9488,8734
2,0,3,0,3,2000,0,0,0,9146,8426
3,0,4,0,3,2000,0,0,0,8687,8451
4,0,5,0,3,2000,0,0,0,8379,8793
...,...,...,...,...,...,...,...,...,...,...
147,18,4,12410,13,12500,0,0,17402,9012,8300
148,18,5,23770,18,17000,2,0,42569,9146,8426
149,18,6,23650,18,17000,1,0,38182,9180,8721
150,18,7,23395,18,17043,5,0,62923,9012,8300


In [10]:
match_details.merge(match_timeline, on='participantId')

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,queueName,minute,totalGold,level,xp,minionsKilled,jungleMinionsKilled,totalDamageDoneToChampions,posX,posY
0,0,0,2,0,0,0,0,12500,13,22,...,Arena,0,0,3,2000,0,0,0,9513,9193
1,0,0,2,0,0,0,0,12500,13,22,...,Arena,1,1000,3,2000,0,0,3634,4085,4201
2,0,0,2,0,0,0,0,12500,13,22,...,Arena,2,1350,4,3500,0,0,4552,8746,9560
3,0,0,2,0,0,0,0,12500,13,22,...,Arena,3,1355,4,3500,0,0,6719,13224,5128
4,0,0,2,0,0,0,0,12500,13,22,...,Arena,4,2705,6,5000,0,0,6719,14636,14181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0,0,4,0,0,0,0,17043,18,55,...,Arena,14,19530,16,15500,1,0,39338,8282,9190
148,0,0,4,0,0,0,0,17043,18,55,...,Arena,15,19880,16,15500,1,0,44129,4240,12731
149,0,0,4,0,0,0,0,17043,18,55,...,Arena,16,19880,18,17028,1,0,51288,14443,12903
150,0,0,4,0,0,0,0,17043,18,55,...,Arena,17,22880,18,17028,1,0,51288,14779,4661
